In [1]:
import pdb

In [2]:
import pandas as pd
import numpy as np
import os

## Variables

In [3]:
THRESHOLD = 10

In [5]:
# 419-->

# orns in fraud_detection

In [95]:
remove = fake_not_detected[fake_not_detected[0].isin(fraud['orn'])]
remove.columns = ['orn']

In [132]:
set1 = set(list(fake_not_detected[0]))

In [133]:
set2 = set(list(remove['orn']))

In [134]:
list1 = list(set1-set2)

In [135]:
from random import sample

In [136]:
orns_1500 = sample(list1,1500)

In [137]:
orns_1500_df = pd.DataFrame(orns_1500)

In [138]:
orns_1500_df.columns = ['orn']

In [139]:
orns_1500_df.to_csv('1500_orns.csv',index=False)

In [140]:
len(fake_not_detected)

12246

In [144]:
sum(orns_1500_df['orn'].isin(fraud["orn"]))

0

In [143]:
pd.Series(["NO0000HG8ZR3"]).isin(fraud["orn"])

0    False
dtype: bool

## Reading

In [6]:
fraud = pd.DataFrame([])
for sheet in os.listdir("../data"):
    if(sheet!='.DS_Store'):
        temp_df = pd.read_csv("../data/"+sheet)
        fraud = pd.concat([fraud,temp_df])
        print('LOADED: ',sheet)

LOADED:  liveness_Fraud_may28.csv
LOADED:  liveness_Fraud_may29.csv
LOADED:  liveness_Fraud_jun3.csv
LOADED:  liveness_Fraud_jun2.csv
LOADED:  liveness_Fraud_jun1.csv
LOADED:  liveness_Fraud_may30.csv
LOADED:  liveness_Fraud_may31.csv
LOADED:  liveness_Fraud_may25.csv
LOADED:  liveness_Fraud_may27.csv
LOADED:  liveness_Fraud_may26.csv


In [7]:
# fraud.to_csv('may25tojune3_liveness_fraud.csv',index=False)

In [8]:
# fraud = pd.DataFrame([])
# for i in range(13,25):
#     temp_df = pd.read_csv(const + str(i) + ".csv")
#     temp_df["date"] = str(i)
#     fraud = pd.concat([fraud,temp_df])

In [9]:
# turing = pd.read_csv('liveness_turingIssues_13_24may.csv')

### NEW CHANGES

In [10]:
turing_df1 = pd.read_csv('turing/MLOPS Liveness May 21 -27.csv')

/Users/vasavilakshmi/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (8,9) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [11]:
turing_df2 = pd.read_csv('turing/MLOPS Liveness May 28 - June 3.csv')

In [12]:
from datetime import datetime

In [13]:
x=turing_df1["created_at"][0]

In [14]:
turing_df1['created_at'] = pd.to_datetime(turing_df1['created_at'], utc=True)

In [15]:
import datetime,pytz

In [16]:
def change_time_zone(df):
    
    df['created_at'] = df['created_at'].dt.tz_convert('Asia/Kolkata')
    return df

In [17]:
turing_df1 = change_time_zone(turing_df1)

In [18]:
turing_df1 = turing_df1[turing_df1['created_at'].apply(lambda x: x.date().day>=25)]

In [19]:
# turing_df1.to_csv('changed_time.csv')

## Manipulation

In [20]:
turing =pd.concat([turing_df1,turing_df2])

In [21]:
turing = turing.rename({'type':'capture_type'},axis='columns')

In [22]:
req_cols = ['xcall_id', 'response_code', 'orn','capture_type', 'is_live','manual_response', 'storeid']

In [23]:
turing.loc[(turing["digitalfake"]==True)|(turing['digitalfake']==True),'manual_response'] = 'fake'

In [24]:
# turing.loc[turing.fakeface==False,'manual_response'] = 'real'
# turing.loc[turing.fakeface==True,'manual_response'] = 'fake'

In [25]:
turing2 = turing[turing['is_live']!='no']

In [26]:
df = pd.concat([fraud[req_cols],turing2[req_cols]])

#### Get Journey status for the following orns

In [27]:
def filter_testing_orn(df):
    df['key'] = df['orn'].apply(lambda x: x[:2])
    filter_df = df[(df['key']=='NO')|(df['key']=='CO')]
    return filter_df

In [28]:
#Removes extra 0 from storeids.
#Returns as string 
def clean_storeid(storeid):
    storeid = str(storeid)
    if len(storeid)<10:
        return storeid
    else:
        return storeid[1:]

In [29]:
df['storeid'] = df['storeid'].apply(lambda x: clean_storeid(x))

In [30]:
df = filter_testing_orn(df)

In [31]:
# fraud = filter_testing_orn(fraud)

In [32]:
# unique_store_id = df[df['manual_response']=='fake']['storeid'].unique()

In [33]:
# output_df = pd.DataFrame(unique_store_id,columns=['store_id'])

In [34]:
new_df = df[df['manual_response']=='fake'].groupby('storeid').agg({'manual_response': 'count', 'orn': ['nunique', 'unique']})

In [35]:
new_df.columns = ['count_of_fakes', 'count_of_faked_orns', 'faked_orns']

In [36]:
new_df.reset_index(inplace=True)

In [37]:
common_stores = new_df.sort_values('count_of_faked_orns',ascending=False)

In [38]:
# new_df = common_stores.sort_values(by='count_of_fakes',ascending=False)

In [39]:
common_stores['storeid'] = common_stores['storeid'].apply(lambda x: clean_storeid(x))

In [40]:
common_stores = common_stores[common_stores['count_of_fakes']>=THRESHOLD]

In [41]:
flagged_stores_1162 = pd.read_csv('flagged stores/Reliance flagged stores - already blacklisted.csv')
flagged_stores_2729 = pd.read_csv('flagged stores/Reliance flagged stores - to blacklist.csv')
flagged_stores_1162.columns = ['storeid']
flagged_stores_2729.columns = ['storeid']

In [42]:
common_stores['is_store_blacklisted (1162)'] = False
common_stores.loc[common_stores['storeid'].isin(flagged_stores_1162['storeid']),'is_store_blacklisted (1162)'] = True

In [43]:
common_stores.isna().sum()

storeid                        0
count_of_fakes                 0
count_of_faked_orns            0
faked_orns                     0
is_store_blacklisted (1162)    0
dtype: int64

##### Save common_stores

In [44]:
common_stores['faked_orns'] = common_stores['faked_orns'].apply(lambda x: str(list(x)))
# common_stores.to_csv('get_journey_status_May25ToJune3.csv',index=False)
common_stores['faked_orns'] = common_stores['faked_orns'].apply(lambda x: eval(x))

##### Read Journey_status

In [45]:
journey_status = pd.read_csv('journey status/journey_status_May25_June3.csv')

In [46]:
journey_status.columns = ['ORN', 'storeid', 'Journey']

#### Check if we got all orns journey status

In [48]:
data = {'ORN':['NO0000HFGSL3', 'NO0000HFHC2V', 'NO0000HHR1C0'],
        'storeid':["660574841", "660574841", "660574841"],
       'Journey':["Complete","Complete","Complete"]}

In [49]:
journey_status = journey_status.append(pd.DataFrame(data)).reset_index().drop('index',axis=1)

In [50]:
journey_status_requested_orns = pd.Series([st for row in common_stores["faked_orns"] for st in row])

In [51]:
journey_status_received_orns = journey_status['ORN']

In [52]:
journey_status_requested_orns[np.logical_not(journey_status_requested_orns.isin(journey_status_received_orns))]

Series([], dtype: object)

In [53]:
missed_data = sum(np.logical_not(journey_status_requested_orns.isin(journey_status_received_orns)))

In [54]:
if missed_data !=0:
    raise Exception("Some of the orns are missing")

In [55]:
missed_data

0

##### As, we have 0 False, it means we got the data for all the orns which we have sent

## Report

In [56]:
#Renaming journey values
journey_status['journey'] = 'Incomplete'
journey_status.loc[journey_status['Journey']=='Complete',"journey"]='Complete'

In [57]:
complete_incomplete_df = journey_status.groupby(['storeid','journey']).agg({'ORN':'unique'}).reset_index()
complete_orns = complete_incomplete_df[complete_incomplete_df['journey']=='Complete']
incomplete_orns = complete_incomplete_df[complete_incomplete_df['journey']=='Incomplete']

total_ORNs_per_store = journey_status.groupby('storeid').agg({'ORN':'unique'}).reset_index()
total_ORNs_per_store.columns = ['storeid', 'total_orns']

def get_intersection(lst1, lst2):
    return list(set(lst1) & set(lst2))

def get_string_list(df,list_cols):
    for col in list_cols:
        df.loc[:,col] = df[col].apply(lambda x: str(list(x)))
    return df


def get_report(df):
    storeid = df['storeid']
    try:
        complete_orns_list = complete_orns[complete_orns['storeid']==storeid]['ORN'].iloc[0]
    except:
        #IF THERE ARE NO COMPLETED ORNs FROM A GIVEN STORE
        complete_orns_list = []
    try:
        incomplete_orns_list = incomplete_orns[incomplete_orns['storeid']==storeid]['ORN'].iloc[0]
    except:
        #IF THERE ARE NO INCOMPLETED ORNs FROM A GIVEN STORE
        incomplete_orns_list = []
        
    df['list_complete_orn'] = complete_orns_list
    df['list_incomplete_orn'] = incomplete_orns_list
    df['count_completed'] = len(complete_orns_list)
    df['count_incompleted']= len(incomplete_orns_list) 
    
    df['list_fake_complete'] = get_intersection(complete_orns_list,df['faked_orns'])
    df['list_fake_incomplete'] = get_intersection(incomplete_orns_list,df['faked_orns'])
    
    df['count_fake_complete'] = len(df['list_fake_complete']) 
    df['count_fake_incomplete'] = len(df['list_fake_incomplete'])
    
    completed_fake_not_detected_list = set(df['list_complete_orn']) - set(df['list_fake_complete']) #Gives real competed
    df['list_completed_fake_not_detected'] = completed_fake_not_detected_list
    df['count_completed_fake_not_detected'] = len(completed_fake_not_detected_list)

    return df

In [58]:
report = common_stores.apply(lambda x: get_report(x),axis=1)
list_cols = ['list_complete_orn', 'list_incomplete_orn','list_fake_complete', 'list_fake_incomplete','list_completed_fake_not_detected']
report = get_string_list(report,list_cols)

## Phase 2

#### Add if store is present in blacklist or not

In [59]:
flagged_stores_1162 = pd.read_csv('flagged stores/Reliance flagged stores - already blacklisted.csv')
flagged_stores_2729 = pd.read_csv('flagged stores/Reliance flagged stores - to blacklist.csv')
flagged_stores_1162.columns = ['storeid']
flagged_stores_2729.columns = ['storeid']

In [60]:
flagged_stores = pd.concat([flagged_stores_2729,flagged_stores_1162])

In [61]:
flagged_stores.nunique()

storeid    3500
dtype: int64

In [62]:
report['Within New Blacklist 1162 Stores'] = False
report.loc[report['storeid'].isin(flagged_stores_1162['storeid']),'Within New Blacklist 1162 Stores'] = True

In [63]:
report['Within New Blacklist 2729 Stores'] = False
report.loc[report['storeid'].isin(flagged_stores_2729['storeid']),'Within New Blacklist 2729 Stores'] = True

In [64]:
report = report.drop(['is_store_blacklisted (1162)'],axis=1)

In [65]:
# report.to_csv('report_may25_june3.csv',index=False)

## Phase 3

#### Report with only required columns

In [66]:
report_cols = ['storeid','Within New Blacklist 1162 Stores', 'Within New Blacklist 2729 Stores',"count_of_fakes",'count_fake_incomplete','count_fake_complete','count_completed_fake_not_detected']
          

In [67]:
final_report = report[report_cols]

In [68]:
final_report.columns = ['storeid','Within New Blacklist 1162 Stores','Within New Blacklist 2729 Stores','Fraudlent attempts','Stopped ORNs','Successful (Fraudulent) ORNs','Successful ORNs with proper data']

In [69]:
sum(final_report['Successful ORNs with proper data'])

12246

In [70]:
# final_report.to_csv('final_report_may25_june3.csv',index=False)`

## Phase 4

#### Get Individual lists of Fake Complete, Fake Incomplete, Fake Not Detected

In [71]:
date = 'may25_june3'

In [72]:
def get_list(df):
    list_orns = [st for row in df for st in row]
    return pd.DataFrame(list_orns)

In [73]:
def get_eval_list(df,list_cols):
    for col in list_cols:
        df[col] = df[col].apply(lambda x: eval(x))
    return df

In [74]:
report = get_eval_list(report,list_cols)

In [75]:
fake_complete_orns = get_list(report['list_fake_complete'])

In [76]:
fake_incomplete_orns = get_list(report['list_fake_incomplete'])

In [77]:
fake_not_detected = get_list(report['list_completed_fake_not_detected'])

In [78]:
# fake_complete_orns.to_csv('fake_complete_orns_' + date + '.csv')

# fake_incomplete_orns.to_csv('fake_incomplete_orns_' + date + '.csv')

# fake_not_detected.to_csv('fake_not_detected_orns_' + date + '.csv')

In [105]:
# may25-June3 Report
# 1162 stores


# friday - 204 stores